Copyright **`(c)`** 2021 Giovanni Squillero `<squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free for personal or classroom use; see 'LICENCE.md' for details.

# Connect 4

In [406]:
from collections import Counter
import numpy as np

In [407]:
NUM_COLUMNS = 7
COLUMN_HEIGHT = 6
FOUR = 4

# Board can be initiatilized with `board = np.zeros((NUM_COLUMNS, COLUMN_HEIGHT), dtype=np.byte)`
# Notez Bien: Connect 4 "columns" are actually NumPy "rows"

## Basic Functions

In [408]:
def valid_moves(board):
    """Returns columns where a disc may be played"""
    return [n for n in range(NUM_COLUMNS) if board[n, COLUMN_HEIGHT - 1] == 0]


def play(board, column, player):
    """Updates `board` as `player` drops a disc in `column`"""
    (index,) = next((i for i, v in np.ndenumerate(board[column]) if v == 0))
    board[column, index] = player


def take_back(board, column):
    """Updates `board` removing top disc from `column`"""
    (index,) = [i for i, v in np.ndenumerate(board[column]) if v != 0][-1]
    board[column, index] = 0


def four_in_a_row(board, player):
    """Checks if `player` has a 4-piece line"""
    return (
        any(
            all(board[c, r] == player)
            for c in range(NUM_COLUMNS)
            for r in (list(range(n, n + FOUR)) for n in range(COLUMN_HEIGHT - FOUR + 1))
        )
        or any(
            all(board[c, r] == player)
            for r in range(COLUMN_HEIGHT)
            for c in (list(range(n, n + FOUR)) for n in range(NUM_COLUMNS - FOUR + 1))
        )
        or any(
            np.all(board[diag] == player)
            for diag in (
                (range(ro, ro + FOUR), range(co, co + FOUR))
                for ro in range(0, NUM_COLUMNS - FOUR + 1)
                for co in range(0, COLUMN_HEIGHT - FOUR + 1)
            )
        )
        or any(
            np.all(board[diag] == player)
            for diag in (
                (range(ro, ro + FOUR), range(co + FOUR - 1, co - 1, -1))
                for ro in range(0, NUM_COLUMNS - FOUR + 1)
                for co in range(0, COLUMN_HEIGHT - FOUR + 1)
            )
        )
    )

## Montecarlo Evaluation

In [409]:
def _mc(board, player):
    p = -player
    while valid_moves(board):
        p = -p
        c = np.random.choice(valid_moves(board))
        play(board, c, p)
        if four_in_a_row(board, p):
            return p
    return 0


def montecarlo(board, player):
    montecarlo_samples = 100
    cnt = Counter(_mc(np.copy(board), player) for _ in range(montecarlo_samples))
    return (cnt[1] - cnt[-1]) / montecarlo_samples


def eval_board(board, player):
    if four_in_a_row(board, 1):
        # Alice won
        return 1
    elif four_in_a_row(board, -1):
        # Bob won
        return -1
    else:
        # Not terminal, let's simulate...
        return montecarlo(board, player)

## Example

In [410]:
board = board = np.zeros((NUM_COLUMNS, COLUMN_HEIGHT), dtype=np.byte)
play(board, 3, 1)
play(board, 0, -1)
play(board, 4, 1)
play(board, 0, -1)
play(board, 5, 1)
play(board, 0, -1)
play(board, 0, -1)
print(board)
eval_board(board, 1)

[[-1 -1 -1 -1  0  0]
 [ 0  0  0  0  0  0]
 [ 0  0  0  0  0  0]
 [ 1  0  0  0  0  0]
 [ 1  0  0  0  0  0]
 [ 1  0  0  0  0  0]
 [ 0  0  0  0  0  0]]


-1

## Homework

In [411]:
from typing import Tuple, List

### Basic Functions

In [412]:
class state_dict:

  def __init__(self):
    self.dict = {}

  def check(board: np.ndarray, player: int):
    # TODO: check dict
    return

  def store(board: np.ndarray, player: int):
    # TODO: store state
    return
  

def who_won(board: np.ndarray) -> int:
  if four_in_a_row(board, 1):
    return 1
  elif four_in_a_row(board, -1):
    return -1
  else:
    return 0


def compute_heuristic(board: np.ndarray) -> float:
  """Value between -1 and 1, computed by counting possible winning configurations for each player"""

  if who_won(board):  # game has ended
    return who_won(board)

  total_count = NUM_COLUMNS * (COLUMN_HEIGHT - FOUR + 1) \
                + COLUMN_HEIGHT * (NUM_COLUMNS - FOUR + 1) \
                + 2 * (NUM_COLUMNS - FOUR + 1) * (COLUMN_HEIGHT - FOUR + 1)
  board1 = np.copy(board) + (board == 0)
  board2 = np.copy(board) + (board == 0)

  return (
    sum(
      all(board1[c, r] == 1) - all(board2[c, r] == -1)
      for c in range(NUM_COLUMNS)
      for r in (list(range(n, n + FOUR)) for n in range(COLUMN_HEIGHT - FOUR + 1))
    )
    + sum(
      all(board[c, r] == 1) - all(board2[c, r] == -1)
      for r in range(COLUMN_HEIGHT)
      for c in (list(range(n, n + FOUR)) for n in range(NUM_COLUMNS - FOUR + 1))
    )
    + sum(
      np.all(board[diag] == 1).astype(np.byte) - np.all(board2[diag] == -1)
      for diag in (
        (range(ro, ro + FOUR), range(co, co + FOUR))
        for ro in range(0, NUM_COLUMNS - FOUR + 1)
        for co in range(0, COLUMN_HEIGHT - FOUR + 1)
      )
    )
    + sum(
      np.all(board[diag] == 1).astype(np.byte) - np.all(board2[diag] == -1)
      for diag in (
        (range(ro, ro + FOUR), range(co + FOUR - 1, co - 1, -1))
        for ro in range(0, NUM_COLUMNS - FOUR + 1)
        for co in range(0, COLUMN_HEIGHT - FOUR + 1)
      )
    )
  ) / total_count

### Minmax

In [413]:
MAX_DEPTH = 4

def minmax(board: np.ndarray, player: int, depth: int) -> Tuple[int, float]:
  """Pick best move for player at each turn looking depth moves ahead"""
  if depth == 0:
    return None, compute_heuristic(board)
  else:
    consider_moves = []  # contains list of tuples (move, heuristic)
    for move in valid_moves(board):
      play(board, move, player)  # make move
      winner = who_won(board)
      if winner == player:  # player wins
        heuristic = compute_heuristic(board)
        take_back(board, move)
        return move, heuristic
      if winner == -player:  # player looses
        take_back(board, move)
        continue
      else:
        _, heuristic = minmax(board, -player, depth-1)
        consider_moves.append((move, heuristic))
        take_back(board, move)
    if len(consider_moves) == 0:  # every valid move results in defeat
      return None, -player
    return max(consider_moves, key=lambda item: player * item[1])  # return best move for player

### Monte Carlo Search

In [414]:
SEARCH_LENGTH = 10
EXPLOITATION_BONUS = 1.2
NUM_SIMULATIONS = 10
HEURISTIC_OFFSET = 0  # ensure that nodes can get picked even if they don't win with default policy

def simulate_random(board: np.ndarray, player: int) -> int:
  """Simulate random moves, return winner"""
  board = np.copy(board)
  while valid_moves(board) and not who_won(board):
    move = np.random.choice(valid_moves(board))
    play(board, move, player)
    player = -player
  return who_won(board)


def run_default_policy(board: np.ndarray, player_max: int, player_turn: int, n: int=NUM_SIMULATIONS) -> float:
  """Run n random simulations and return win percentage for player_max plus offset"""
  num_wins = 0
  for _ in range(NUM_SIMULATIONS):
    if simulate_random(board, player_turn) == player_max:
      num_wins += 1
  return (num_wins + HEURISTIC_OFFSET) / (NUM_SIMULATIONS + HEURISTIC_OFFSET)


def expand_node(board: np.ndarray, player: int, moves: List[int]) -> List[Tuple[List[int], float]]:
  """Get original board and player along with list of moves leading to node we are expanding."""
  player_max = player
  player_turn = player
  board = board.copy()
  children = []
  for move in moves:  # set board to state of current node
    play(board, move, player_turn)
    player_turn = -player_turn
  if who_won(board):  # don't expand terminal nodes
    return [(moves, run_default_policy(board, player_max, player_turn))]
  for move in valid_moves(board):  # for each child
    play(board, move, player_turn)
    if who_won(board) == -player_max:  # opponent wins
      return []  # discard this subtree as opponent can always win
    elif who_won(board) == player_max:  # player wins
      return [(moves + [move], run_default_policy(board, player_max, -player_turn))]  # this move results in victory
    child = (moves + [move], run_default_policy(board, player_max, -player_turn))
    children.append(child)
    take_back(board, move)
  return children
  

def montecarlo_sample(nodes: List[Tuple[List[int], float]]) -> List[Tuple[List[int], float]]:
  """Pick node to expand and remove from list"""
  compute_weight = lambda node: node[1] * EXPLOITATION_BONUS ** len(node[0])  # heuristic * EXPLOITATION_BONUS ** depth
  weights = [compute_weight(node) for node in nodes]  # compute weights
  p = [w / sum(weights) for w in weights]  # normalize
  idx_choice = np.random.choice(len(nodes), p=p)
  return nodes.pop(idx_choice)  # remove node from list and return it


def montecarlo_search(board: np.ndarray, player: int, search_length: int=SEARCH_LENGTH) -> int:
  nodes = [([], run_default_policy(board, player, player))]  # contains list of tuples (list of moves, heuristic)
  for _ in range(search_length):
    if len(nodes) == 0:  # usually occurs when no options left
      return np.random.choice(valid_moves(board))
    next_node = montecarlo_sample(nodes)
    nodes += expand_node(board, player, next_node[0])
  return (montecarlo_sample(nodes)[0])[0]

### Play Game

In [ ]:
def play_human(board: np.ndarray, player: int):
  print(board)
  move = -1
  while move not in valid_moves(board):
    move = int(input())
  play(board, move, player)


def play_minmax(board: np.ndarray, player: int):
  move, prediction = minmax(board, -1, MAX_DEPTH)
  if prediction == -player:  # game is already lost
    move = np.random.choice(valid_moves(board))
  play(board, move, player)


def play_montecarlo(board: np.ndarray, player: int):
  move = montecarlo_search(board, player)
  play(board, move, player)


def play_game():
  """Play vs the computer"""
  board = np.zeros((NUM_COLUMNS, COLUMN_HEIGHT), dtype=np.byte)
  while len(valid_moves(board))>0 and not who_won(board):
    play_montecarlo(board, 1)
    play_human(board, -1)
  print(f'Player {who_won(board)} won!')
  print(board)

play_game()